In [1]:
from pyomo.environ import *

In [2]:
model=ConcreteModel()
model.x_1=Var(within=NonNegativeReals)
model.x_2=Var(within=NonNegativeReals)
model.obj=Objective(expr=model.x_1+2*model.x_2)
model.con1=Constraint(expr=3*model.x_1+4*model.x_2>=1)
model.con2=Constraint(expr=2*model.x_1+5*model.x_2>=2)

In [3]:
N=[1,2]
M=[1,2]
c={1:1,2:2}
a={(1,1):3,(1,2):4,(2,1):2,(2,2):5}
b={1:1,2:2}

In [5]:
model=ConcreteModel()
model.x=Var(N,within=NonNegativeReals)

def obj_rule(model):
    return sum(c[i]*model.x[i] for i in N)
model.obj=Objective(rule=obj_rule)

def con_rule(model,m):
    return sum(a[m,i]*model.x[i] for i in N)>=b[m]
model.con=Constraint(M,rule=con_rule)

In [10]:
model=AbstractModel()

model.N=Set()
model.M=Set()
model.c=Param(model.N)
model.a=Param(model.M,model.N)
model.b=Param(model.M)

model.x=Var(model.N,within=NonNegativeReals)
def obj_rule(model):
    return sum(model.c[i]*model.x[i] for i in model.N)
model.obj=Objective(rule=obj_rule)

def con_rule(model,m):
    return sum(model.a[m,i]*model.x[i] for i in model.N)
model.con=Constraint(model.M,rule=con_rule)

In [16]:
vertices=set(['Ar','Bo','Br','Ch','Co','Ec','Fg','Gu','Pa','Pe','Su','Ur','Ve'])

edges=set([('Fg','Su'),('Fg','Br'),('Su','Gu'),
           ('Su','Br'),('Gu','Ve'),('Gu','Br'),
           ('Ve','Co'),('Ve','Br'),('Co','Ec'),
           ('Co','Pe'),('Co','Br'),('Ec','Pe'),
           ('Pe','Ch'),('Pe','Bo'),('Pe','Br'),
           ('Ch','Ar'),('Ch','Bo'),('Ar','Ur'),
           ('Ar','Br'),('Ar','Pa'),('Ar','Bo'),
           ('Ur','Br'),('Bo','Pa'),('Bo','Br'),
           ('Pa','Br')])
ncolors=4
colors=range(1,ncolors+1)

model=ConcreteModel()

model.x=Var(vertices,colors,within=Binary)
model.y=Var()

model.node_coloring=ConstraintList()
for v in vertices:
    model.node_coloring.add(sum(model.x[v,c] for c in colors)==1)
    
model.edge_coloring=ConstraintList()
for v,w in edges:
    for c in colors:
        model.edge_coloring.add(model.x[v,c]+model.x[w,c]<=1)

model.min_coloring=ConstraintList()
for v in vertices:
    for c in colors:
        model.min_coloring.add(model.y>=c*model.x[v,c])

model.obj=Objective(expr=model.y)